## **Portfolio Rehab:** An automated portfolio risk analyzer w/ stock pic recomendations

Create a Jupyter notebook that outputs

## **Pitch:**
- You are a long term invester who prefers to buy and hold positions 
- You like your positions, but are aware of the risks associated with the elevated prices in a the current frothy market environment
- You want insight into portfolio risks, and stategies to mitigate or hedge said risk to facilitate quality long term returns
- You do not want to pay for expensive, monthly programs and would rather invest in a one time analysis report of your portfolio

## **Inputs:**
* Stock portfolio: tickers, holding volume, average price
* Time horizon 
* Risk tolerance (Low / near retirement or sell point, medium / typical invester with stable income. high / higher income expected in the future)
* Desired average annual return over the time horizon
* % current margin use
* Avoidance options: No fossil fuels, no China stocks, etc.

# **Risk analysis components:**

### **Correlation analysis**
#### Inputs
- Pull time 5yr price time series, or the longest length in the portfolio
    * Note: This will not work as well for IPOs... exclude outliers more than X Stds from the next shortest time series (mention in output)   
    
#### Outputs
- Output matrix into report
- Associate correlation coefficient ranges to risk label, and recomendations
    * What % of equity is highly correlated, % somewhat correlated, % not correlated, % inversely correlated, % strongly inversely correlated
- Save arrays and use for recomendation algorithm at the end

### **Intrest rates risk analysis**
#### Inputs
- Calculate P/E, P/S, CAPE distribution in the portoflio
- Calculate averages weighted by equity for the above values 
- Pull historic P/E, P/S, and CAPE where available and calculate correlation with intrest rates / liquidity 

#### Outputs
- To what degree are position valuations negatively correlated with US treasury intrest rates
    * How does correlation coefficient and R^2 compare to S&P broadly 
    * Describe comparison to market as well as correlation qualititively in a string output
    * Make graphs showing P/E, P/S overlayed w/ intrest rates
- To what degree are average valuations predicted to mean revert, compare to sector means as well as market means via standard deviations

### **Discounted Cash Flow (DCF) analysis**
Here we apply Sven's discounted cash flow model using the

#### Inputs
- User: Desired annualized return / discount rate
- User: Time horizon (T)
- Historic terminal multiple series -- Used to calculate probability of different terminal multiple scenarios
- Recent earnings (quaterly and annuals) 
- Anything else in Sven's model...

Fit decay curve (exponential) to annual revenue and earnings growth to estimate future growth rates within time horizon

#### Outputs
- Estimated fair value for desired return after T in optimistic, middle, and pessimistic scenarios. As well as weighted average.
- If estimated returns are lower that expected (i.e. value after DCF analysis is less tha





In [57]:
# Import dependencies
import matplotlib.pyplot as plt
import scipy 
from scipy.optimize import curve_fit
import scipy.stats as stats
import pandas as pd
import numpy as np

import FundamentalAnalysis as fa
import analysis_functions
from analysis_functions import *

### Load in net income data for a given ticker

In [59]:
ticker = "AAPL"
api_key = "7dead7d8a15b90bbadab8eeedfa59bd0"
last_year = 2020

grab_data = False

if grab_data == True:
    cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
    cash_flow = cash_flow_statement_annually.transpose()
    cash_flow = cash_flow.rename_axis('year')
    cash_flow.to_csv(r'C:\Users\xavie\Portfolio_Rehab\test_data\%s_cash_flow.csv' % ticker)
    
else:
    cash_flow = pd.read_csv(r'C:\Users\xavie\Portfolio_Rehab\test_data\%s_cash_flow.csv' % ticker)

revenue_np = cash_flow['netIncome'].to_numpy()

### Gain insight into 10 yr historical income growth (update to work with shorter life stocks)

In [71]:
%matplotlib widget
x = np.arange(last_year, last_year - 10, -1)
y = revenue_np[:10]

x_inc = x[::-1][1:] # Years increasing
growth = []

for i in range(len(y) - 1):
    rate = ((y[i] - y[i + 1]) / y[i + 1]) * 100
    growth.append(rate)

growth_inc = growth.reverse() # Growth rate by year increasing   
growth_array = np.array(growth)
mean_growth = np.mean(growth_array)

# Plot
plt.plot(x_inc, growth_array)
plt.hlines(mean_growth, xmin=x_inc[0], xmax=x_inc[-1], label='Mean growth rate percent = %s' % round(mean_growth, 2))
plt.xlabel('Year')
plt.ylabel('% annual revenue growth from previous year')
plt.grid()
plt.legend()

first = growth_array[:5].mean()
second = growth_array[5:].mean()
print('Annual growth mean for %s to %s: %s' % (x_inc[0], x_inc[5], first))
print('Annual growth mean for %s to %s: %s' % (x_inc[5], x_inc[-1], second))


drops = ((first - second) / first)
drops_percent = drops * 100
print('5 year growth reduction percent: %s' % drops_percent )

future_rates = [second * drops, second * drops**2] # [0-5 year growth rate, 5-10 year growth rate]
print(future_rates)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Annual growth mean for 2012 to 2017: 15.425081467765938
Annual growth mean for 2017 to 2020: 6.418114895858314
5 year growth reduction percent: 58.3916953095492
[3.7476460946063774, 2.1883140888427755]


## **Discounted cash flow valuation model** 

Upload historic p/e data csv and pull current quote

In [80]:
# Test w/ AAPL data!
pe_data = r'C:\Users\xavie\Portfolio_Rehab\test_data\aapl_pe_test.csv'

# Pull up to date stock values
quote_df = fa.quote(ticker, api_key)
quote_df = quote_df.transpose()

price = quote_df['price'].to_numpy()[0]
market_cap = quote_df['marketCap'].to_numpy()[0]
shares = market_cap / price

print('Current %s price: %s' % (ticker, price))

Current AAPL price: 145.86


In [81]:
def discounted_cash_value(cash_flow, discount_rate, time_horizon, growth_rates, pe_data, pe_scenarios, optimism='low'):
    """ This function automated Sven Carlin's discounted cash flow and outputs an intrinsic value based on optimism
    and a modeled growth rate curve.
    Inputs: Current cash flow, discount rate aka desired annualized return (float, percent), time_horizon in years (int),
    a curve of length len(time_horizon) modeling future growth rates, an array storing historic P/E ratios that is used
    to calculate the range of pe_values, optimism (str) either low, medium, or high (effects scenario probabilities)"""

    opt_strs = []

    # Define scenario outcome probabilities for terminal multiple
    if optimism == 'low':
        probs = [0.3, 0.6, 0.1]  # Worst case, medium case, best case

    elif optimism == 'medium':
        probs = [0.2, 0.6, 0.2]  # Worst case, medium case, best case

    elif optimism == 'high':
        probs = [0.1, 0.6, 0.3]  # Worst case, medium case, best case

    else:
        print('Optimism parameter error: must be low, medium, or high')

    # Generate a forecasted growth rate curve
    scenarios = ['pessimistic', 'medium', 'optimistic']
    
    # Calculate instrinsic value for each scenario
    values = [] # Stores the value for each scenario
    
    for i, scene in enumerate(scenarios):
        future_flow = [] # Stores the estimate future cash flows for each scenario
        future_pv = [] # Stores estimate PV values for each scenario
    
        term_pe = pe_scenarios[i]
        
        for j, t in enumerate(range(0, time_horizon)):
            if j <= 4:
                rate = growth_rates[0]
            else:
                rate = growth_rates[1]
                
            if scene is 'optimistic':
                rate = rate * 2
            
            if j == 0:
                future_val = cash_flow * (1 + (rate / 100))
                
            else:
                future_val = future_flow[j - 1] * (1 + (rate / 100))

            future_flow.append(future_val)
            
            exp = -t - 1
            pv = future_val * ((1 + discount_rate)**(exp))
            future_pv.append(pv)
        
        terminal_value = future_flow[-1] * term_pe
        term_pv = terminal_value * ((1 + discount_rate)**(exp))
        future_pv.append(term_pv)
        
        values.append(sum(future_pv)) # FIGURE 
        
    # Calculate weighted instrinsic value by optimism level
    final_val = 0
    for i, value in enumerate(values):
        final_val += (value * probs[i])
    
    # Returns a list storing the optimism weighted average instrinsic value, and another list storing each scenario value
    return [final_val, values] 

In [82]:
if pe_data[-4:] == '.csv':
        pe_df = pd.read_csv(pe_data)
        pe_array = pe_df['pe_ratio'].to_numpy()

else:
    pe_array = pe_data

print('Historic P/E value array:')
print(pe_array)

mean_val = np.nanmean(pe_array)
std_val = np.nanstd(pe_array)

optimist = mean_val + (2 * std_val)
pessimist = mean_val - std_val

scenario_pe = [pessimist, mean_val, optimist]

print()
print('Pessimistic, medium, and optimistic terminal multiple values')
print(scenario_pe)

Historic P/E value array:
[38.23 35.95 27.37 35.82 35.35 27.55 19.76 22.92 18.62 16.56 15.62 12.63
 18.45 16.23 15.6  16.68 16.01 15.58 15.93 13.09 12.84 10.43 11.26 10.32
 10.99 13.26 14.02 13.51 14.16 13.5  11.47 12.37 10.64  8.7   9.22 10.47
 13.04 11.8  12.57  9.92 11.85 11.43 14.28 15.49 16.1  17.54 19.87 20.92
 21.22 17.62 13.31 11.97 18.24 28.17 25.54 37.76 34.   30.08 25.61 26.53]

Pessimistic, medium, and optimistic terminal multiple values
[10.195359855957339, 18.099499999999995, 33.907780288085306]


In [83]:
%matplotlib widget

test_rates = [i for i in range(1, 15)]
scenarios = ['low', 'medium', 'high']
desired_rate = 0.05
print('Desired annualized return is %s percent' % round(float(desired_rate * 100), 2))
labels = ['Pessimistic', 'Neutral', 'Optimistic']
prices = []

for i, opt in enumerate(scenarios):
    comps = []
    price_sub = []
    for initial in test_rates:
        rates = [initial, initial / 2]
        out = discounted_cash_value(revenue_np[0], desired_rate, 10, rates, pe_data, scenario_pe, optimism=opt)
        i_price = float(out[0] / shares) # Intrinsic value share price
        price_sub.append(i_price)
        comp = ((out[0] - market_cap) / aapl_mc) * 100
        comps.append(comp)

    comps_np = np.asarray(comps)
    initial_np = np.asarray(test_rates)
    plt.plot(initial_np, comps_np, label=labels[i])
    plt.legend()
    plt.grid()
    plt.hlines(0, xmin=np.min(initial_np), xmax=np.max(initial_np))
    plt.xlabel('Initial growth rate %, halves after 5 years')
    plt.ylabel('Intrinsic value percent +/- current value for return')
    plt.xlim(np.min(initial_np), np.max(initial_np))
    
    prices.append(price_sub)


# Make plot again and figure out how to make a clean git commit.

Desired annualized return is 5.0 percent


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [84]:
%matplotlib widget

for i, sub in enumerate(prices):
    price_np = np.asarray(sub)

    plt.plot(initial_np, price_np, label=labels[i])
    if i == 1:
        plt.hlines(price, xmin=np.min(initial_np), xmax=np.max(initial_np), label='Current price')
    plt.legend()
    plt.grid()
    plt.xlabel('Initial growth rate %, halves after 5 years')
    plt.ylabel('Fair price for %s percent annual return' % round(float(desired_rate * 100), 2))
    plt.xlim(np.min(initial_np), np.max(initial_np))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## **Intrest rates analysis**

In [87]:
# Load in ticker historic P/E ratio data, and flip from macro axis format
pe_df_inc = pe_df.reindex(index=pe_df.index[::-1])
pe_df_inc.head()

,Date,pe_ratio
59,12/31/2006,26.53
58,3/31/2007,25.61
57,6/30/2007,30.08
56,9/30/2007,34.00
55,12/31/2007,37.76


In [85]:
# Load in historic US Treasury intrest rates
intrest_df = pd.read_csv(r'C:\Users\xavie\Portfolio_Rehab\test_data\intrest_rates.csv')
intrest_df.head()

,DATE,rate
0,1/1/1980,12.00
1,2/1/1980,12.52
2,3/1/1980,13.00
3,4/1/1980,13.00
4,5/1/1980,12.94
